In [9]:
# This cell is NOT editable. Overwrite variables on your own discretion.
# Any changes other than the script code will NOT BE SAVED!
# All cells are assumed to be script code cells, unless explictly tagged as 'o9_ignore'

connect_str = "DefaultEndpointsProtocol=https;AccountName=khazeus2o9devdsmlstg001;AccountKey=+4OpyWQ4ka65EPThoCs4uIvsqkufeBrHiSKQ7vXXQHCh+L6dc0gr0SWnaTi8H/PvRqdLyl1KFVWoy2OixusLwA==;EndpointSuffix=core.windows.net"
destination_name = "prophesy-preprod-ads"
ADS = "Yes"
Pickle = "Yes"
Folder_name = "BatchRun/"
Number_Of_Weeks = "116"
RunType = "Forward"
Parquet_subfolder_prefix = "GenerationWeek_"
Pickle_subfolder_prefix = "ML_Training_models_"

In [10]:
# Slice Dimension Attributes defined in the plugin. Please check all queries and replace <KEY HERE> with a valid name.
# For example: If slice is defined by Version.[Version Name] and Time.[Month]
# input_df = ibpl Select ([Version].[Version Name].[<KEY HERE>] * [Time].[Month].[<KEY HERE>] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;
#                             update <KEY HERE> to valid names
# input_df = ibpl Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Month].[January] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;

_shipment_df = "Select ([Version].[Version Name].[CurrentWorkingView] * &AllPastWeeks * [Sales Domain].[Customer Group] *  [Sales Domain].[SalesOrg_DC] * [Item].[L6].[154] * [Item].[L2]) on row, ({Measure.[Actual Shifted]})  on column ;"
_calendar_df = "Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Week] * [Sales Domain].[Country] ) on row, ({Measure.[Holiday Type], Measure.[Is Holiday]}) on column;"
_temp_planitem_df = "Select ([Item].[L2] * [Item].[Item].filter(#.[Is New Item] != true).relatedmembers([Planning Item])* [Item].[L6].[154]);"
_current_week_df = "Select (&CurrentWeek.element(0) * [Version].[Version Name]);"
_imputed_price_promo_df = "Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Week] * [Sales Domain].[Customer Group] * [Item].[Planning Item]* [Item].[L6].[154] * [Sales Domain].[SalesOrg_DC]* Item.[L2]) on row,  ({Measure.[IRI Distribution Imputed], Measure.[IRI Display Only Price Imputed (lbs)], Measure.[IRI Feature Only Price Imputed (lbs)], Measure.[IRI Feature and Display Price Imputed (lbs)], Measure.[IRI Price Reduction Imputed (lbs)], Measure.[IRI Std Price Imputed (lbs)]}) on column;"
_time_df = "select ([Time].[Day] * [Time].[Week]);"
_compass_ppg = "Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Week] * [Sales Domain].[Customer Group] * [Item].[L2]* [Item].[L6].[154] * [Sales Domain].[SalesOrg_DC]) on row,  ({Measure.[Compass Display Only Price Imputed (EA)], Measure.[Compass Feature Only Price Imputed (EA)], Measure.[Compass Feature and Display Price Imputed (EA)], Measure.[Compass Price Reduction Imputed (EA)], Measure.[Compass Std Price Imputed (EA)],Measure.[US Compass Dist Imputed]}) on column;"
_uom_df = "Select ([UOM].[UOM] * [Version].[Version Name].[CurrentWorkingView] * [Item].[Planning Item] ) on row,  ({Measure.[UOM Conversion]}) on column;"
_weeklist_bt_df = "Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Week] ) on row,  ({Measure.[Week Flag]}) on column;"


# Initialize the O9DataLake with the input parameters and dataframes
# Data can be accessed with O9DataLake.get(<Input Name>)
# Overwritten values will not be reflected in the O9DataLake after initialization

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
O9DataLake.register("shipment_df",DataSource.LS, ResourceType.IBPL, _shipment_df)
O9DataLake.register("calendar_df",DataSource.LS, ResourceType.IBPL, _calendar_df)
O9DataLake.register("temp_planitem_df",DataSource.LS, ResourceType.IBPL, _temp_planitem_df)
O9DataLake.register("current_week_df",DataSource.LS, ResourceType.IBPL, _current_week_df)
O9DataLake.register("imputed_price_promo_df",DataSource.LS, ResourceType.IBPL, _imputed_price_promo_df)
O9DataLake.register("time_df",DataSource.LS, ResourceType.IBPL, _time_df)
O9DataLake.register("compass_ppg",DataSource.LS, ResourceType.IBPL, _compass_ppg)
O9DataLake.register("uom_df",DataSource.LS, ResourceType.IBPL, _uom_df)
O9DataLake.register("weeklist_bt_df",DataSource.LS, ResourceType.IBPL, _weeklist_bt_df)

In [11]:
shipment_df = O9DataLake.get("shipment_df")
calendar_df = O9DataLake.get("calendar_df")
temp_planitem_df = O9DataLake.get("temp_planitem_df")
current_week_df = O9DataLake.get("current_week_df")
imputed_price_promo_df = O9DataLake.get("imputed_price_promo_df")
time_df = O9DataLake.get("time_df")
compass_ppg = O9DataLake.get("compass_ppg")
uom_df = O9DataLake.get("uom_df")
weeklist_bt_df = O9DataLake.get("weeklist_bt_df")

2022-10-07 22:22:57,928 : aabhaschandra_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/ibplquery/6602/ExecuteCompactJsonQuery]
2022-10-07 22:23:00,721 : aabhaschandra_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/ibplquery/6602/ExecuteCompactJsonQuery]
2022-10-07 22:23:00,968 : aabhaschandra_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/ibplquery/6602/ExecuteCompactJsonQuery]
2022-10-07 22:23:02,506 : aabhaschandra_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/ibplquery/6602/ExecuteCompactJsonQuery]
2022-10-07 22:23:02,681 : aabhaschandra_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/ibplquery/6602/ExecuteCompactJsonQuery]
2022-10-07 22:23:02,856 : aabhaschandra_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/ibplquery/6602/ExecuteCompactJsonQuery]
2022-10-07 22:23:03,461 : aabhasch

In [14]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import logging
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from o9_common_utils.O9DataLake import O9DataLake
from functools import reduce
import io
import os, uuid, sys
from azure.storage.blob import BlobServiceClient
import time
import pyarrow as pa
import pyarrow.parquet as pq
import shap

logger = logging.getLogger("o9_logger")
pd.set_option('display.max_columns', 50)

try:
    import pickle5 as picl
except:
    logger.debug("pickle5 Not available")

### Data Import ###
# Getting data from local directory if environment is local
current_path = os.path.dirname(os.path.realpath(__file__))

if current_path.split(":")[0] in ["c", "C", "d", "D"]:
    print('local environment')
    data_subpath = '\\package\\o9helpers\\o9_data_folder'
    names = os.listdir(path=current_path + data_subpath)
    for name in names:
        exec(name + " = pd.read_csv(r'" + current_path + data_subpath + "\\" + name + "\\000')")

else:
    logger = logging.getLogger("o9_logger")
    logger.debug('tenant environment')
    tenant_name = [i for i in sys.path if i.find('site-packages')!=-1][0].split('/')[4].replace('_','/')
    logger.debug("Current Tenant name == "+ tenant_name)

logger.debug("shipment_df : {}".format(shipment_df.shape))
logger.debug("calendar_df : {}".format(calendar_df.shape))

shipment_df = shipment_df.sort_values(
    ['Sales Domain.[Customer Group]', 'Sales Domain.[SalesOrg_DC]', 'Item.[L2]', 'Time.[Week]'])
calendar_df = calendar_df.sort_values(['Time.[Week]'])
time_df = time_df.sort_values(['Time.[Week]'])

### Getting version and category
current_version = shipment_df['Version.[Version Name]'].unique()[0]
cat = shipment_df['Item.[L6]'].unique()[0]

logger.debug("current_version - {}".format(current_version))
logger.debug("cat - {}".format(cat))

### Column handling
### Renaming cols
shipment_df_rename = {
    'Time.[Week]': 'WEEK_NUM',
    'Sales Domain.[Customer Group]': 'Planning_Entity',
    'Sales Domain.[SalesOrg_DC]': 'SalesOrg',
    'Item.[L2]': 'PPG',
    'Actual Shifted': 'Actual'}

uom_df_rename = {
    'UOM.[UOM]': 'UOM',
    'Item.[Planning Item]': 'BASECODE'
}

compass_ppg_rename = {
    'Initiative.[Initiative]': 'Initiative',
    'Time.[Week]': 'WEEK_NUM',
    'Sales Domain.[Customer Group]': 'Planning_Entity',
    'Sales Domain.[SalesOrg_DC]': 'SalesOrg',
    'Item.[L2]': 'PPG',
    'Compass Display Only Price Imputed (EA)': 'Display Only Price',
    'Compass Feature Only Price Imputed (EA)': 'Feature Only Price',
    'Compass Feature and Display Price Imputed (EA)': 'Feature and Display Price',
    'Compass Price Reduction Imputed (EA)': 'Price Reduction Price',
    'Compass Std Price Imputed (EA)': 'Standard Price',
    'US Compass Dist Imputed': 'Distribution'
}

imputed_price_promo_df_rename = {
    'Time.[Week]': 'WEEK_NUM',
    'Sales Domain.[Customer Group]': 'Planning_Entity',
    'Sales Domain.[SalesOrg_DC]': 'SalesOrg',
    'Item.[L2]': 'PPG',
    'Item.[Planning Item]': 'BASECODE',
    'Initiative.[Initiative]': 'Initiative',
    'IRI Display Only Price Imputed (lbs)': 'Display Only Price',
    'IRI Feature Only Price Imputed (lbs)': 'Feature Only Price',
    'IRI Feature and Display Price Imputed (lbs)': 'Feature and Display Price',
    'IRI Price Reduction Imputed (lbs)': 'Price Reduction Price',
    'IRI Std Price Imputed (lbs)': 'Standard Price',
    'IRI Distribution Imputed': 'Distribution'}

temp_compass_df_rename = {
    'Time.[Week]': 'WEEK_NUM',
    'Sales Domain.[Customer Group]': 'Planning_Entity',
    'Sales Domain.[SalesOrg_DC]': 'SalesOrg',
    "Item.[L2]": "PPG",
    'Initiative.[Initiative]': 'Initiative',
    'Standard Price Compass EA 445': 'Standard Price'}

temp_planitem_df_rename = {'Item.[Planning Item]': 'BASECODE', "Item.[L2]": "PPG"}
         
                                                                                              

time_df_rename = {'Time.[Week]': 'WEEK_NUM'}

shipment_df.rename(columns=shipment_df_rename, inplace=True)
imputed_price_promo_df.rename(columns=imputed_price_promo_df_rename, inplace=True)
temp_planitem_df.rename(columns=temp_planitem_df_rename, inplace=True)
calendar_df.rename(columns=time_df_rename, inplace=True)
time_df.rename(columns=time_df_rename, inplace=True)
compass_ppg.rename(columns=compass_ppg_rename, inplace=True)
uom_df.rename(columns=uom_df_rename, inplace=True)
                                                  

                                              
                                            
                                 
                               

# Dropping cols
shipment_df_drop = ['Version.[Version Name]', 'Item.[L6]']
imputed_price_promo_df_drop = ['Version.[Version Name]', 'Item.[L6]']
temp_planitem_df_drop = ['Item.[L6]']
calendar_df_drop = ['Version.[Version Name]']
time_df_drop = ['Version.[Version Name]']
compass_ppg_drop = ['Version.[Version Name]', 'Item.[L6]']
uom_df_drop = ['Version.[Version Name]']
                                        

shipment_df.drop(shipment_df_drop, axis=1, inplace=True)
imputed_price_promo_df.drop(imputed_price_promo_df_drop, axis=1, inplace=True)
temp_planitem_df.drop(temp_planitem_df_drop, axis=1, inplace=True)
calendar_df.drop(calendar_df_drop, axis=1, inplace=True)
uom_df.drop(uom_df_drop, axis=1, inplace=True)
                                                    
compass_ppg.drop(compass_ppg_drop, axis=1, inplace=True)

# Week format preprocessing
for df in [shipment_df, compass_ppg,
           imputed_price_promo_df, time_df, calendar_df]:
    df['WEEK_NUM'] = df['WEEK_NUM'].astype(str)
    df['WEEK_NUM'] = df['WEEK_NUM'].str.replace('-W', '')

# time_df preprocessing to extract month
time_df['Time.[Day]'] = pd.to_datetime(time_df['Time.[Day]'])
time_df = time_df.sort_values(['WEEK_NUM','Time.[Day]'],ascending=True)
time_df = time_df.groupby(['WEEK_NUM'])['Time.[Day]'].first().reset_index()
time_df['Month'] = time_df['Time.[Day]'].dt.month
time_df.drop('Time.[Day]',axis=1,inplace=True)
time_df['Year'] = time_df['WEEK_NUM'].apply(lambda x: x[:-2])
time_df['Week'] = time_df['WEEK_NUM'].apply(lambda x: x[-2:])

### Defining and setting dtypes
shipment_df_dtypes = {
    "WEEK_NUM": np.int64,
    "Planning_Entity": "str",
    "SalesOrg": "str",
    "PPG": "str",
    "Actual": "float"}

imputed_price_promo_df_dtypes = {
    "WEEK_NUM": np.int64,
    "Planning_Entity": "str",
    "SalesOrg": "str",
    "PPG": "str",
    "BASECODE": np.int64}

temp_planitem_df_dtypes = {
    "PPG": "str",
    "BASECODE": np.int64
}

calendar_df_dtypes = {
    "WEEK_NUM": np.int64
}

time_df_types = {
    "WEEK_NUM": np.int64,
    "Year": np.int64,
    "Week": np.int64,
    "Month": np.int64
}

uom_df_types = {
    "BASECODE": np.int64,
}

compass_ppg_types = {
    "PPG": "str",
    "WEEK_NUM": np.int64,
    "SalesOrg": "str",
    "Planning_Entity": "str",
}

                
                         
                      
                              
 

shipment_df = shipment_df.astype(shipment_df_dtypes)
imputed_price_promo_df = imputed_price_promo_df.astype(imputed_price_promo_df_dtypes)
temp_planitem_df = temp_planitem_df.astype(temp_planitem_df_dtypes)
time_df = time_df.astype(time_df_types)
calendar_df = calendar_df.astype(calendar_df_dtypes)
uom_df = uom_df.astype(uom_df_types)
compass_ppg = compass_ppg.astype(compass_ppg_types)
                                    

                                                                             
                                                                                                                                                                
    
                                                 
    
                              
                                                                          
                                     
    
                               
                                                                                                                                                  
                                                        

                                 
                           
                                                                      
                                 
                                   
    
    
                                            
    

                                               
                               
    
                                                                          
                                                                                       
                                                                                                                                                 
    
                                            
                                    
                                                                                                                        
                                        
    
                                                         
                                                                             

                           
                                                                  
                                                                  
                                                                  
        
                                        
                                                
    
                          
                                                                                

        
                                                      
             

def feat_imp(model, train_x1, current_version, ship_max_week, featflag, cat):
    feat = model.feature_importances_
    col = train_x1.columns
    feat_df = pd.DataFrame({'Columns': col, 'Importance': feat})

    feat_data = feat_df.T
    feat_data.columns = feat_data.iloc[0]
    feat_data = feat_data[1:]
    feat_data = feat_data.reset_index(drop=True)

    feat_data['Holiday'] = 0
    feat_data['Base'] = 0
    feat_data['Promotion'] = 0

    logger.debug('Initial feat_data:')
    logger.debug(feat_data)

    allocated_cols = ['Distribution', 'DisplayOnlyPrice',
     'FeatureOnlyPrice', 'FeatureandDisplayPrice', 'PriceReductionPrice', 'StandardPrice']

    for i in feat_data.columns:
        if str(i).startswith("holiday"):
            feat_data['Holiday'] = feat_data["Holiday"] + feat_data[i]
            allocated_cols.append(i)
        #if str(i).startswith("US"):
            #feat_data['Base'] = feat_data["Base"] + feat_data[i]
            #allocated_cols.append(i)            
        #if len(str(i))==15:
            #feat_data['Base'] = feat_data["Base"] + feat_data[i]
            #allocated_cols.append(i)
        if 'FLAG' in str(i):
            feat_data['Promotion'] = feat_data["Promotion"] + feat_data[i]
            allocated_cols.append(i)
            

    feat_data['Time Trend'] = feat_data['Week'] + feat_data['Month']
    allocated_cols.append('Week'); allocated_cols.append('Month'); 
    
    feat_data['Seasonality'] = feat_data['mnth_cos'] + feat_data['mnth_sin'] + feat_data['week_cos'] + feat_data[
        'week_sin']
    allocated_cols.append('mnth_cos'); allocated_cols.append('mnth_sin'); 
    allocated_cols.append('week_cos'); allocated_cols.append('week_sin'); 

    created_cols = ['Time Trend', 'Seasonality','Promotion','Holiday','Base']    
    unallocated_cols = list(set(feat_data.columns) - set(allocated_cols) - set(created_cols))

    for i in unallocated_cols:
        feat_data['Base'] = feat_data["Base"] + feat_data[i]

    if featflag:
        feat_data = feat_data[
            ['Holiday', 'Base', 'Promotion', 'Time Trend', 'Seasonality', 'Distribution', 'DisplayOnlyPrice',
             'FeatureOnlyPrice', 'FeatureandDisplayPrice', 'PriceReductionPrice', 'StandardPrice']]
        feat_data.rename(columns={'DisplayOnlyPrice': 'Display Only Price',
                                  'FeatureOnlyPrice': 'Feature Only Price',
                                  'FeatureandDisplayPrice': 'Feature and Display Price',
                                  'PriceReductionPrice': 'Price Reduction Price', 'StandardPrice': 'Standard Price'},
                         inplace=True)
    else:
        feat_data = feat_data[['Holiday', 'Base', 'Time Trend', 'Seasonality']]

    feat_data['Version.[Version Name]'] = current_version
    feat_data['Time.[Week]'] = ship_max_week
    feat_data['Item.[L6]'] = cat

    feat_data['Sales Domain.[Sales Org]'] = 'US01'

    feat_data['Time.[Week]'] = feat_data['Time.[Week]'].map(lambda x: str(x)[:4] + "-W" + str(x)[4:])

    feat_data['Item.[L6]'] = feat_data['Item.[L6]'].map(lambda x: '0' + str(x) if len(str(x)) == 2 else str(x))

                                                 

    feat_data = feat_data.melt(id_vars=['Version.[Version Name]', 'Time.[Week]', 'Item.[L6]','Sales Domain.[Sales Org]'],
                               var_name='[ML Driver].[ML Driver]', value_name='Values')
    feat_data.rename(columns={'Values': 'Driver Importance'}, inplace=True)

    feat_data['Driver Importance'] = feat_data['Driver Importance'].astype(float)



    if "CatBoost" in str(type(model)):
        feat_data.rename(columns={'Driver Importance':'Driver Importance Model 1'}, inplace=True)
        feat_data['Driver Importance Model 1'] = feat_data['Driver Importance Model 1'] / 100
    if "LGBM" in str(type(model)):
        feat_data.rename(columns={'Driver Importance':'Driver Importance Model 2'}, inplace=True)
        importance_sum = feat_data['Driver Importance Model 2'].sum()
        if importance_sum!=0:
            feat_data['Driver Importance Model 2'] = feat_data['Driver Importance Model 2']/importance_sum
        else:
            feat_data['Driver Importance Model 2'] = 0
    if "RandomForest" in str(type(model)):
        feat_data.rename(columns={'Driver Importance':'Driver Importance Model 3'}, inplace=True)
    if "XGB" in str(type(model)):
        feat_data.rename(columns={'Driver Importance':'Driver Importance Model 4'}, inplace=True)
    return feat_data

def upload_ads_to_azure(source_df, azure_connection_string, destination_file_name, destination_name):
    """
    Function to upload data from contents to an Azure container
    :source_df: data frame name that needs to be transferred

    :param azure_connection_string: .
    :param destination_file_name:
    :param destination:
    :param azure_container_name:
    :return:
    """
    logger.debug('Trying to uploading contents of {} to {} located in Azure container {}.'.format("source_df",
                                                                                                  destination_name + "/ " + destination_file_name,
                                                                                                  destination_name))
    try:
        blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)

        output_par = io.BytesIO()
        table = pa.Table.from_pandas(source_df)
        pq.write_table(table, output_par)
        output_par.seek(0)
        blob_client = blob_service_client.get_blob_client(container=destination_name,
                                                          blob=destination_file_name + ".parquet")
        if blob_client.exists():
            blob_client.delete_blob()
        blob_client.upload_blob(output_par)

        return True
    except:
        logging.exception("Azure upload to {} failed.".format(destination_name + "/ " + destination_file_name))
        return False


def upload_pickle_to_azure(source_pickle, azure_connection_string, destination_file_name, destination_name):
    logger.debug('Trying to uploading contents of {} to {} located in Azure container {}.'.format("source_df",
                                                                                                  destination_name + "/ " + destination_file_name,
                                                                                                  destination_name))
    try:
        blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
        blob_client = blob_service_client.get_blob_client(container=destination_name, blob=destination_file_name)
        if blob_client.exists():
            blob_client.delete_blob()
        output = io.BytesIO()
        picl.dump(source_pickle, output)
        output.seek(0)
        blob_client.upload_blob(output)

        return True
    except:
        logging.exception("Azure upload to {} failed.".format(destination_name + "/ " + destination_file_name))


def ads(shipment_df, imputed_price_promo_df, year_week_df, calendar_df, uom_df, compass_ppg):
    #### Processing shipment data ####

    # Column "Shipment_Volume" consists of values (0,+ve values)
    shipment_df.loc[shipment_df['Actual'] < 0, 'Shipment_Volume'] = 0
    shipment_df['Shipment_Volume'].fillna(shipment_df['Actual'], inplace=True)
    logger.debug(shipment_df.shape)
    logger.debug("shipment_df - Shipment_Volume column and value")
    logger.debug(shipment_df.head())

    shipment_df = shipment_df.groupby(['PPG', 'Planning_Entity', 'WEEK_NUM', 'SalesOrg'])[
        'Shipment_Volume'].sum().reset_index()
    if RunType=='Forward':
        current_week_df['Time.[Week]'] = current_week_df['Time.[Week]'].str.replace('-W', '')
        current_week = current_week_df['Time.[Week]'].unique()[0]
    elif RunType=='Backtest':
        weeklist_bt_df['Time.[Week]'] = weeklist_bt_df['Time.[Week]'].str.replace('-W', '')
        current_week = weeklist_bt_df['Time.[Week]'].unique()[0]
    current_week = int(current_week)
    logger.debug("Current week of the forecast")
    logger.debug(current_week)

    logger.debug("shipment_df shape")
    logger.debug(shipment_df.shape)
    # Creating ship max week after filtering
    ship_max_idx = year_week_df[time_df['WEEK_NUM'] == current_week].index - 1
    ship_max_week = year_week_df.loc[ship_max_idx, 'WEEK_NUM'].values[0]
    logger.debug("ship_max_week")
    logger.debug(ship_max_week)

    # creating ship min week to filter out
    ship_min_week = shipment_df['WEEK_NUM']
    ship_min_week = ship_min_week.min()
    logger.debug("Shipment min week num")
    logger.debug(ship_min_week)

    Train_Periods = 156
    training_start_week_index = year_week_df[year_week_df['WEEK_NUM'] == current_week].index - Train_Periods #!! Should be current_week
    starting_week = year_week_df.loc[training_start_week_index, 'WEEK_NUM'].values[0]

    filter_week_index = year_week_df[year_week_df['WEEK_NUM'] == current_week].index - 54 #!! Should be current_week
    filter_week = year_week_df.loc[filter_week_index, 'WEEK_NUM'].values[0]

    # Creating ending_week = current_week + Number_Of_Weeks + Max number of weeks to get lead features (4 for holidays) TODO: check if necessary
    ending_week_index = year_week_df[year_week_df['WEEK_NUM'] == current_week].index + (int(Number_Of_Weeks)) + 4 #!! Should be current_week
    ending_week = year_week_df.loc[ending_week_index, 'WEEK_NUM'].values[0]

    iri_max_week_index = time_df[time_df['WEEK_NUM'] == current_week].index - 4
    iri_max_week = time_df.loc[iri_max_week_index, 'WEEK_NUM'].values[0]

    compass_start_week_index = iri_max_week_index + 1
    compass_start_week = time_df.loc[compass_start_week_index, 'WEEK_NUM'].values[0]

    # filtering out current week shipment
    shipment_df = shipment_df[shipment_df['WEEK_NUM'] < current_week]

    # obtain the df of valid intersections to fcst in the future (intersections with data in past 54 wks)
    map_df = shipment_df[shipment_df['WEEK_NUM'] >= filter_week][
        ['PPG', 'Planning_Entity', 'SalesOrg']].drop_duplicates()
    
    # Filtering out all combinations not present in last 54 weeks
    shipment_df = pd.merge(map_df, shipment_df, on=['PPG', 'Planning_Entity', 'SalesOrg'], how='left')

    # For fcst intersections that have weeks where there is no shipment, we want to create rows with zero shipment
    # get the weeks for which we are considering shipments
    time_df_map = year_week_df[(year_week_df['WEEK_NUM'] >= starting_week) &
                               (year_week_df['WEEK_NUM'] <= ship_max_week)][['WEEK_NUM']]
    # get all fcst intersections which have occurred in the past
    # shipment_df_map = shipment_df[['PPG', 'Planning_Entity', 'SalesOrg']].drop_duplicates()
    map_df['key'] = 0
    time_df_map['key'] = 0

    # perform a cross join of fcst intersections and time so that  there is a week for each fcst intersection
    # Example: WEEK_NUM x PPG            x Planning_Entity x SalesOrg   Shipment_Volume
    #          202130     02300910021600F  US4000415         US01-20    5472
    #          202135     02300910021600F  US4000415         US01-20    11.4
    # becomes
    #          WEEK_NUM x PPG            x Planning_Entity x SalesOrg   Shipment_Volume
    #          202129     02300910021600F  US4000415         US01-20    nan    
    #          202130     02300910021600F  US4000415         US01-20    5472
    #          202131     02300910021600F  US4000415         US01-20    nan
    #          202132     02300910021600F  US4000415         US01-20    nan
    #          202133     02300910021600F  US4000415         US01-20    nan
    #          202134     02300910021600F  US4000415         US01-20    nan
    #          202135     02300910021600F  US4000415         US01-20    11.4
    shipment_df_temp = pd.merge(time_df_map, map_df, on='key', how='left')
    shipment_df_temp.drop('key', axis=1, inplace=True)
    map_df.drop('key', axis=1, inplace=True)

    # join that tmp ship df back on the original ship df. Any originally missing weeks for intersections will be NaN
    shipment_df = pd.merge(shipment_df_temp, shipment_df, on=['PPG', 'Planning_Entity', 'WEEK_NUM', 'SalesOrg'],
                           how='left')

    # for each forecast intersection, get the first week for which there was actuals
    first_shipments = shipment_df[shipment_df['Shipment_Volume'] > 0].groupby(['PPG', 'Planning_Entity', 'SalesOrg'])[
        'WEEK_NUM'].first().reset_index()
    first_shipments.rename({'WEEK_NUM': 'first_week'}, axis=1, inplace=True)

    # bring that first shipment week column into our shipment df and filter to remove weeks before first shipment
    # Example: WEEK_NUM x PPG            x Planning_Entity x SalesOrg   Shipment_Volume
    #          202128     02300910021600F  US4000415         US01-20    nan
    #          202129     02300910021600F  US4000415         US01-20    nan
    #          202130     02300910021600F  US4000415         US01-20    5472
    # where 202130 is the first shipment volume for this combination, is replaced by
    #          WEEK_NUM x PPG            x Planning_Entity x SalesOrg   Shipment_Volume
    #          202130     02300910021600F  US4000415         US01-20    5472    
    # and all later weeks

    # bring that first shipment week column into our shipment df and filter to remove weeks before first shipment
    shipment_df = pd.merge(shipment_df, first_shipments, on=['PPG', 'Planning_Entity', 'SalesOrg'])
    shipment_df = shipment_df[(shipment_df['WEEK_NUM']) >= (shipment_df['first_week'])]
    shipment_df.drop('first_week', axis=1, inplace=True)

    # Processing Sales Driver data #
    # Creating flag for categories w/o driver data
    FSFlag = True if imputed_price_promo_df.shape[0] == 0 else False
    logger.debug("FS Flag")
    logger.debug(FSFlag)

    # IRI Price
    if not FSFlag:
        logger.debug("Not FSFlag")
        # Compass Processing
        # IRI start/end week and compass start/end week
        compass_ppg = compass_ppg[(compass_ppg.WEEK_NUM >= compass_start_week) & (compass_ppg.WEEK_NUM <= ending_week)]

        # Considering only the price columns for mapping to basecode level with temp_planitem_df (Only prices require UoM conversion)
        compass_df = pd.merge(compass_ppg.drop('Distribution',axis=1), temp_planitem_df, on='PPG', how='left')
        
        uom_df = uom_df[uom_df['UOM'] == 'EA']
        compass_df = pd.merge(compass_df, uom_df, on='BASECODE', how='left').drop_duplicates()
        compass_df.loc[compass_df['UOM Conversion'].isnull(), 'UOM Conversion'] = 1

        compass_df.rename(columns={'Compass Distribution Imputed': 'Distribution'}, inplace=True)

        compass_df = (compass_df.groupby(by=['Planning_Entity', 'PPG', 'WEEK_NUM', 'SalesOrg'], observed=True)[
                          ['Display Only Price', 'Feature Only Price',
                           'Feature and Display Price',
                           'Price Reduction Price',
                           'Standard Price',
                           'UOM Conversion'
                           ]]
                      .mean().reset_index())

        compass_df = pd.merge(compass_df, compass_ppg[['WEEK_NUM', 'Planning_Entity', 'SalesOrg',
                                                                  'Distribution', 'PPG']],
                              on=['WEEK_NUM', 'Planning_Entity', 'SalesOrg', 'PPG'])


        # Converting using conversion factor
        compass_df['Display Only Price'] = compass_df['Display Only Price'] * compass_df['UOM Conversion']
        compass_df['Feature Only Price'] = compass_df['Feature Only Price'] * compass_df['UOM Conversion']
        compass_df['Feature and Display Price'] = compass_df['Feature and Display Price'] * compass_df['UOM Conversion']
        compass_df['Price Reduction Price'] = compass_df['Price Reduction Price'] * compass_df['UOM Conversion']
        compass_df['Standard Price'] = compass_df['Standard Price'] * compass_df['UOM Conversion']

        compass_df.drop(['UOM Conversion'], axis=1, inplace=True)

        # IRI Processing
        # Price/Promotion data preprocessing- (not necessary for TDP data) #
        # imputed_price_promo_df = imputed_price_promo_df[
            # [col for col in imputed_price_promo_df.columns if 'Compass' not in col]]
        imputed_price_promo_df = imputed_price_promo_df[(imputed_price_promo_df.WEEK_NUM >= starting_week) &
                                                        (imputed_price_promo_df.WEEK_NUM <= iri_max_week)]
        
        imputed_price_promo_df['IRI_flag'] = np.where(imputed_price_promo_df['Standard Price']>0, 1, 0)

        # IRI_flag is next aggregated over all the basecodes under each ppg
        # Taking a single PPG and WEEK_NUM for example
        # Example: WEEK_NUM x BASECODE    x Planning_Entity x SalesOrg x PPG             .... x IRI_flag
        #          201901     430000294500  US3000059         US01-20    023011300312XF8        1
        #          201901     430000294500  US4000150         US01-20    023011300312XF8        1
        #          201901     430000294500  US4000071         US01-20    023011300312XF8        1
        #          201901     430000294500  US4000151         US01-20    023011300312XF8        1
        #          201901     430000294500  US2000013         US01-20    023011300312XF8        1
        #          201901     430000294500  US3000030         US01-20    023011300312XF8        1
        #          201901     430000294500  US4000153         US01-20    023011300312XF8        1
        #          201901     430000294800  US2000013         US01-20    023011300312XF8        1
        #          201901     430000294900  US2000013         US01-20    023011300312XF8        1
        #          201901     430000294900  US3000030         US01-20    023011300312XF8        1
        #          201901     430000295100  US4000071         US01-20    023011300312XF8        1
        #          201901     430000295100  US3000030         US01-20    023011300312XF8        1
        #          201901     430007086500  US2000013         US01-20    023011300312XF8        1
        #          201901     430007086500  US3000059         US01-20    023011300312XF8        1
        #          201901     430007086500  US4000153         US01-20    023011300312XF8        1
        #          201901     430007086500  US3000030         US01-20    023011300312XF8        1

        agg_by_dict = {'Display Only Price': 'mean', 
                       'Feature Only Price': 'mean',
                       'Feature and Display Price': 'mean',
                       'Price Reduction Price': 'mean',
                       'Standard Price': 'mean', 
                       'Distribution': 'mean',
                       'IRI_flag': 'sum'}
        imputed_price_promo_df = (imputed_price_promo_df.groupby(['WEEK_NUM', 'PPG', 'Planning_Entity', 'SalesOrg'])
                                  ['Standard Price', 'Display Only Price', 'Feature Only Price',
                                      'Feature and Display Price', 'Price Reduction Price', 'Distribution','IRI_flag']
                                  .agg(agg_by_dict).reset_index())

        # After aggregation, 
        # Example: WEEK_NUM x Planning_Entity x SalesOrg x PPG             .... x IRI_flag
        #          201901     US3000059         US01-20    023011300312XF8        2
        #          201901     US4000150         US01-20    023011300312XF8        1
        #          201901     US4000071         US01-20    023011300312XF8        2
        #          201901     US4000151         US01-20    023011300312XF8        1
        #          201901     US2000013         US01-20    023011300312XF8        4
        #          201901     US3000030         US01-20    023011300312XF8        4
        #          201901     US4000153         US01-20    023011300312XF8        2
        
                                                                                  
                                                                                      

        #Setting up a mapping dataframe using IRI_flag from imputed_price_promo_df  
        #This helps propagate IRI_flag to the Compass (future) part of sales_driver_df
        sdmap_df = imputed_price_promo_df[['PPG', 'Planning_Entity', 'SalesOrg', 'IRI_flag']].drop_duplicates()
        imputed_price_promo_df.drop('IRI_flag',axis=1,inplace=True) 

        logger.debug('imputed_price_promo_df shape')
        logger.debug(imputed_price_promo_df.shape)
        logger.debug('imputed_price_promo_df columns')
        logger.debug(imputed_price_promo_df.columns)

                                                                                                      

        sales_driver_df = pd.concat([imputed_price_promo_df, compass_df], axis=0) #!!Is this sufficient?
        
        sales_driver_df = pd.merge(sdmap_df, sales_driver_df, on=['PPG', 'Planning_Entity', 'SalesOrg'], how='left')

        logger.debug("Sales driver shape")
        logger.debug(sales_driver_df.shape)
        logger.debug("Sales driver describe")
        logger.debug(sales_driver_df.describe())

    else:
        logger.debug("No sales driver processing possible")

    # Merging shipment with sales drivers #
    # Creating all combinations of basecode, planning entity & salesorg that are present in last 54 weeks
    time_df_map = year_week_df[(year_week_df['WEEK_NUM'] >= current_week) &
                               (year_week_df['WEEK_NUM'] <= ending_week)][['WEEK_NUM']] #!!Brings in only WEEK_NUM, which is correct

    # Cross join - Each combination is being mapped to each week in the entire 156 weeks + next 114 weeks
    map_df['key'] = 1
    time_df_map['key'] = 1
    future_records = pd.merge(time_df_map, map_df, on='key')
    future_records.drop(columns='key', inplace=True)

    # Stack the shipment history with the future intersections to forecast
    main_df = pd.concat([shipment_df, future_records], axis=0)

    main_df = main_df[['WEEK_NUM', 'PPG', 'Planning_Entity', 'SalesOrg', 'Shipment_Volume']]
    logger.debug("Filtering nulls weeks before starting the shipment")
    logger.debug(main_df.shape)
    logger.debug(main_df.head())
    
                              
                                                                       
                                                                               
    
                                             
                                          
    
                                                                                  
                                                                                      

    # Merging shipment with sales drivers #
    if not FSFlag:
        logger.debug("not FSFlag - 1")

        main_df = pd.merge(main_df, sales_driver_df, on=['WEEK_NUM', 'PPG', 'Planning_Entity', 'SalesOrg'],
                           how='left').drop_duplicates()

        # Example
        # WEEK_NUM x PPG              x Planning_Entity x SalesOrg    Shipment_Volume    x IRI_flag    
        # 201927     023009100216JE8    US4000576          US01-20    510                  3            
        # 201928     023009100216JE8    US4000576          US01-20    nan                  3            
        # cont..
        # Display Only Price x Feature Only Price x Feature and Display Price x Price Reduction Price x Standard Price x Distribution
        # 7.346199989           7.346199989            7.346199989                    7.036043326                7.346199989         84.4221
        # 7.333030065           7.333030065            7.333030065                    5.380936623                7.333030065         83.01346667

        for col in ['Display Only Price', 'Feature Only Price', 'Feature and Display Price', 'Price Reduction Price']:
            main_df[col + ' ratio'] = (main_df['Standard Price'].round(2) - main_df[col].round(2)) / main_df[
                'Standard Price'].round(2)
            main_df[col] = main_df[col + ' ratio'].apply(lambda x: abs(100 * x)).round(2)
            main_df[col].fillna(0, inplace=True)
            main_df.drop([col + ' ratio'], axis=1, inplace=True)

        # After transformation becomes
        # WEEK_NUM x PPG              x Planning_Entity x SalesOrg    Shipment_Volume    x IRI_flag    
        # 201927     023009100216JE8    US4000576          US01-20    510                  3            
        # 201928     023009100216JE8    US4000576          US01-20    nan                  3            
        # cont..
        # Display Only Price x Feature Only Price x Feature and Display Price x Price Reduction Price x Standard Price x Distribution
        # 0                       0                    0                            4.22                    7.346199989         84.4221
        # 0                       0                    0                            26.6                    7.333030065         83.01346667    

        # Promotion Measures- If the promo price feature is > 0, then the value is flagged as 1 (promotion flag)
        main_df['DISPLAY_ONLY_PROMO_FLAG'] = np.where(main_df['Display Only Price'] > 0, 1, 0)
        main_df['FEATURE_DISPLAY_PROMO_FLAG'] = np.where(main_df['Feature and Display Price'] > 0, 1, 0)
        main_df['FEATURE_ONLY_PROMO_FLAG'] = np.where(main_df['Feature Only Price'] > 0, 1, 0)
        main_df['PRICE_REDUCTIONS_ONLY_PROMO_FLAG'] = np.where(main_df['Price Reduction Price'] > 0, 1, 0)

        main_df = main_df.sort_values(['WEEK_NUM'])

        # Creating lead1 and lead2 features for Promotions
        # Note- lead1 corresponds to shift(-2) due to KHC convention of treating current week as lag -1
        promo_col = [col for col in main_df if col.endswith('FLAG')]
        # Eg. For a combination, row of W35, Lead1 will have the flag value of week 37 if there is a shift of -2 #!!Change to lead1
        for col in promo_col:
            main_df[col + '_lead2'] = main_df.groupby(['PPG', 'Planning_Entity', 'SalesOrg'])[col].shift(-2)
            main_df[col + '_lead3'] = main_df.groupby(['PPG', 'Planning_Entity', 'SalesOrg'])[col].shift(-3)

    else:
        logger.debug("FSFlag is true")
        # Setting IRI_flag
        main_df['IRI_flag'] = 0
        logger.debug("main_df shape after setting IRI_flag")
        logger.debug(main_df.shape)
        logger.debug("main_df head")
        logger.debug(main_df.head())

    # Extraction of Holiday Calendar #
    calendar_df = calendar_df[calendar_df['Sales Domain.[Country]'] == 'US']

    # Creating a matrix of all holidays as column and then allocation 'no of holidays' for a specific week
    calendar_df = calendar_df.pivot_table(index=['WEEK_NUM'], columns='Holiday Type', values='Is Holiday',
                                          aggfunc='mean', fill_value=0).reset_index()
    calendar_df = calendar_df.rename_axis(None, axis=1)

    # Renaming holiday columns
    for i in calendar_df.columns:
        if 'WEEK' not in i:
            calendar_df.rename(columns={i: 'holiday_' + i}, inplace=True)
    logger.debug("Holiday shape")
    logger.debug(calendar_df.shape)

    main_df = pd.merge(main_df, calendar_df, on='WEEK_NUM', how='left').drop_duplicates()

    # Creating lead1 and lead2 features for Holiday
    holiday_col = [col for col in main_df if col.startswith('holiday')]

    # Eg. For a combination, row of W35, Lead1 will have the flag value of week 37 if there is a shift of -2
    # Note- lead1 corresponds to shift(-2) due to KHC convention of treating current week as lag -1
    main_df = main_df.sort_values(['WEEK_NUM'])
    for col in holiday_col:
        main_df[col + '_lead2'] = main_df.groupby(['PPG', 'Planning_Entity', 'SalesOrg'])[col].shift(-2) #!! Should be calling it lead1
        main_df[col + '_lead3'] = main_df.groupby(['PPG', 'Planning_Entity', 'SalesOrg'])[col].shift(-3)
        main_df[col + '_lead4'] = main_df.groupby(['PPG', 'Planning_Entity', 'SalesOrg'])[col].shift(-4)

    logger.debug("main df shape after holiday merging")
    logger.debug(main_df.shape)

    # Creating Week, Year and Month columns- Week and Month are used as features #
    main_df = pd.merge(main_df, year_week_df, on='WEEK_NUM', how='left')

    # Creating cyclic (trigonometric) features #
    # Creating sin and cos month cyclic values
    main_df['mnth_sin'] = np.sin((main_df.Month - 1) * (2. * np.pi / 12))
    main_df['mnth_cos'] = np.cos((main_df.Month - 1) * (2. * np.pi / 12))
    # creating sin and cos weekly cyclic values
    main_df['week_sin'] = np.sin((main_df.Week - 1) * (2. * np.pi / 52))
    main_df['week_cos'] = np.cos((main_df.Week - 1) * (2. * np.pi / 52))

    # Example
    # WEEK_NUM x PPG              x Planning_Entity    x SalesOrg x Shipment_Volume x Month x Year x Week x mnth_sin x mnth_cos x week_sin    x week_cos
    # 201905     0230113003123QH    US4000071          US01-20      0                   1       2019      5         0            1           0.464723172     0.885456026

    logger.debug("Shape of Main data before splitting into ML and MLL")
    logger.debug(main_df.shape)

    # Partitioning for driver (ML) and driverless (MLL) parts of the ADS #
    # data for ML model
    ml_df = main_df[main_df['IRI_flag'] >= 1]
    ml_df.drop(['IRI_flag'], axis=1, inplace=True)

    logger.debug("Shape of ml_df")
    logger.debug(ml_df.shape)

    # data for MLL model
    mll_df = main_df[main_df['IRI_flag'].fillna(0) == 0]
    mll_df.drop(['IRI_flag'], axis=1, inplace=True)

    logger.debug("Shape of mll_df")
    logger.debug(mll_df.shape)

    # removing space and punctuations from column names
    ml_df = ml_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))
    mll_df = mll_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

    return ml_df, mll_df, starting_week, ship_max_week, ship_min_week


def model_results(forecast_list, current_version):
    # Generating o9 output #
    df_results = pd.concat(forecast_list, ignore_index=True)

    # Shape of result before EPM merging
    logger.debug("Shape of result before EPM merging")
    logger.debug(df_results.shape)

    # converting to o9 format and adding EPM column
    df_results = df_results[['Time.[Week]', 'PPG', 'Planning_Entity', 'SalesOrg', 'Predicted']]
    df_results['PPG'] = df_results['PPG'].astype(str)
    df_results['Planning_Entity'] = df_results['Planning_Entity'].astype(str)
    df_results['SalesOrg'] = df_results['SalesOrg'].astype(str)
    df_results.rename(columns={'PPG': 'Item.[L2]'}, inplace=True)
    df_results.rename(columns={'Planning_Entity': 'Sales Domain.[Customer Group]'}, inplace=True)
    df_results['Version.[Version Name]'] = current_version
    df_results['Time.[Week]'] = df_results['Time.[Week]'].map(lambda x: str(x)[:4] + "-W" + str(x)[4:])
    df_results = df_results[
        ['Version.[Version Name]', 'Item.[L2]', 'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg',
         'Predicted']]
    df_results.loc[df_results['Predicted'] <= 0, 'Predicted'] = 0

    # Shape of result
    logger.debug("Shape of result after renaming")
    logger.debug(df_results.shape)

    return df_results


def model_run(ml_df, mll_df, starting_week, ship_min_week, ship_max_week, current_version, cat):
    if len(ml_df) != 0:
        logger.debug("ML Model is running")
        logger.debug(ml_df.shape)

        ml_df.fillna(0, inplace=True)
        ml_df = ml_df.sort_values(by=['PPG', 'Planning_Entity', 'SalesOrg', 'WEEK_NUM']).reset_index(drop=True)

        remove = ['Shipment_Volume', 'PPG',
                  'WEEK_NUM', 'Planning_Entity', 'SalesOrg', 'PRICE_REDUCTIONS_ONLY_PROMO_FLAG',
                  'DISPLAY_ONLY_PROMO_FLAG',
                  'FEATURE_DISPLAY_PROMO_FLAG', 'FEATURE_ONLY_PROMO_FLAG', 'weekno', 'weights', 'Year']


        partial_remove = ['Shipment_Volume', 'PRICE_REDUCTIONS_ONLY_PROMO_FLAG',
                  'DISPLAY_ONLY_PROMO_FLAG',
                  'FEATURE_DISPLAY_PROMO_FLAG', 'FEATURE_ONLY_PROMO_FLAG', 'weekno', 'weights', 'Year']

        if RunType == 'Forward':
            weeklist = sorted(i for i in ml_df['WEEK_NUM'].unique().tolist() if i > ship_max_week)[:-4]
        elif RunType == 'Backtest':
            weeklist = [sorted(i for i in ml_df['WEEK_NUM'].unique().tolist() if i > ship_max_week)[3]]

        logger.debug("Weeks to be forecasted")
        logger.debug(weeklist)

        forecasted_xgb = []
        forecasted_cat = []
        forecasted_lgb = []
        forecasted_rfm = []

        ml_df['WEEK_NUM'] = ml_df['WEEK_NUM'].astype(np.int64)
        train = ml_df[(ml_df['WEEK_NUM'] >= starting_week) & (ml_df['WEEK_NUM'] <= ship_max_week)]
        test = ml_df[(ml_df['WEEK_NUM'] > ship_max_week)]

        train['IsTrain'] = 1
        test['IsTrain'] = 0
        #######
        ML_ADS = pd.concat([train, test], axis=0, ignore_index=True)
        train = train.drop(['IsTrain'], axis=1)
        test = test.drop(['IsTrain'], axis=1)
        ######

        logger.debug("train and test shapes")
        logger.debug(train.shape)
        logger.debug(test.shape)

        ls1 = train['Planning_Entity'].unique().tolist()
        test = test[test['Planning_Entity'].isin(ls1)]

        ls2 = train['PPG'].unique().tolist()
        test = test[test['PPG'].isin(ls2)]

        dummy1 = pd.get_dummies(train['PPG'])
        dummy2 = pd.get_dummies(train['Planning_Entity'])
        # dropping zero columns
        try:
            dummy1.drop([0], axis=1, inplace=True)
            dummy2.drop([0], axis=1, inplace=True)
        except:
            logger.debug("No zero column in dummy variables")

        train = pd.concat([train, dummy1, dummy2], axis=1)

        train.replace(np.inf, np.nan, inplace=True)
        train.fillna(0, inplace=True)

        dummy1 = pd.get_dummies(test['PPG'])
        dummy2 = pd.get_dummies(test['Planning_Entity'])
        # dropping zero columns
        try:
            dummy1.drop([0], axis=1, inplace=True)
            dummy2.drop([0], axis=1, inplace=True)
        except:
            logger.debug("No zero column in dummy variables")

        test = pd.concat([test, dummy1, dummy2], axis=1)

        test.replace(np.inf, np.nan, inplace=True)
        test.fillna(0, inplace=True)

        cols = test.columns.to_list()
        train = train[cols]

        # creating recency based model weights on training set
        le = preprocessing.LabelEncoder()
        logger.debug('Encoder input:')
        logger.debug(train.WEEK_NUM.values)
        train['weekno'] = le.fit_transform(train.WEEK_NUM.values)
        logger.debug('Encoder output:')
        logger.debug(train['weekno'].head())
        train["weights"] = 30 + (train["weekno"] - train["weekno"].max()) * np.exp(-2)
        train["weights"] = train["weights"].astype(float)
        train['weights'] = train['weights'] / train['weights'].max()
        train.loc[(train.WEEK_NUM > 202005) & (train.WEEK_NUM < 202028), 'weights'] = 0.2
        train.loc[(train.WEEK_NUM > 202010) & (train.WEEK_NUM < 202020), 'weights'] = 0.05

        train_y1 = train['Shipment_Volume']
        train_x1 = train[train.columns[~train.columns.isin(remove)]]

        # 1 Xgboost Model
        xgboostr = XGBRegressor(random_state=2017)
        xgboostr.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        logger.debug('Xgboost Fitting completed successfully')

        # 2 Catboost Model
        cbr = CatBoostRegressor(random_state=2017)
        cbr.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        # 3 lgbm Model
        lgbr = LGBMRegressor(random_state=2017)
        lgbr.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        # 4 rfm Model
        rfm = RandomForestRegressor(random_state=2017)
        rfm.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        models = {'xgboostr': xgboostr, 'cbr': cbr, 'lgbr': lgbr, 'rfm': rfm}

        logger.debug('Fitting completed successfully')

        try:
            explainer_cbr = shap.TreeExplainer(cbr)
            explainer_lgbr = shap.TreeExplainer(lgbr)
            explainer_rfm = shap.TreeExplainer(rfm)
            explainer_xgboostr = shap.TreeExplainer(xgboostr)

            explainers = {'xgboostr': explainer_xgboostr, 'cbr': explainer_cbr, 'lgbr': explainer_lgbr, 'rfm': explainer_rfm}


            start = time.time()
            upload_pickle_to_azure(explainers, connect_str, tenant_name+ '/' + Folder_name + Pickle_subfolder_prefix + str(
                ship_max_week) + '/explainers_' + str(cat) + '_US'+'.pkl', destination_name)
            logger.debug('time taken to write in pickle file : ' + str(time.time() - start))

            test_x = test[test.columns[~test.columns.isin(partial_remove)]]

            start = time.time()
            upload_ads_to_azure(test_x, connect_str,
                                tenant_name + '/' + Folder_name + Parquet_subfolder_prefix + str(ship_max_week) + '/ML_fwd_' + str(cat) + '_US_test_ML',
                                destination_name)
            logger.debug('time taken to write in parquet format : ' + str(time.time() - start))
            
        except:
            logger.debug('explainer failed to generate')


        predict_start_time = time.time()

        logger.debug(weeklist)
        for i in weeklist:
            logger.debug('i')
            logger.debug(i)

            test_1 = test[test['WEEK_NUM'] == i]

            test_y1 = test_1['Shipment_Volume']
            test_x1 = test_1[test_1.columns[~test_1.columns.isin(remove)]]

            train_x1 = train_x1.fillna(0)
            test_x1 = test_x1.fillna(0)

            cols = train_x1.columns.to_list()
            test_x1 = test_x1[cols]

            logger.debug("week: {}".format(i))
            # XGBoost prediction
            pred_xgb = xgboostr.predict(test_x1)
            pred_xgb[pred_xgb < 0] = 0

            testdata_xgb = test_1.copy()
            testdata_xgb['Predicted'] = pred_xgb
            testdata_xgb['Time.[Week]'] = i
            testdata_xgb = testdata_xgb[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_xgb.append(testdata_xgb)

            # Catboost prediction
            pred_cat = cbr.predict(test_x1)
            pred_cat[pred_cat < 0] = 0

            testdata_cat = test_1.copy()
            testdata_cat['Predicted'] = pred_cat
            testdata_cat['Time.[Week]'] = i
            testdata_cat = testdata_cat[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_cat.append(testdata_cat)

            # lgbm prediction
            pred_lgb = lgbr.predict(test_x1)
            pred_lgb[pred_lgb < 0] = 0

            testdata_lgb = test_1.copy()
            testdata_lgb['Predicted'] = pred_lgb
            testdata_lgb['Time.[Week]'] = i
            testdata_lgb = testdata_lgb[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_lgb.append(testdata_lgb)

            # Random forest prediction
            pred_rfm = rfm.predict(test_x1)
            pred_rfm[pred_rfm < 0] = 0

            testdata_rfm = test_1.copy()
            testdata_rfm['Predicted'] = pred_rfm
            testdata_rfm['Time.[Week]'] = i
            testdata_rfm = testdata_rfm[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_rfm.append(testdata_rfm)
                
        logger.debug('Predictions generated successfully')

        predict_end_time = time.time()
        predict_time = predict_end_time - predict_start_time
        logger.debug('predict_time: {}'.format(predict_time))

        logger.debug('Generated weeks: {}'.format(testdata_rfm['Time.[Week]'].unique().tolist()))

        # Generating Xgboost output###
        df_results_xgb = model_results(forecasted_xgb, current_version)
        # Generating Catboost output###
        df_results_cat = model_results(forecasted_cat, current_version)
        # Generating LGBM output###
        df_results_lgb = model_results(forecasted_lgb, current_version)
        # Generating RFM output###
        df_results_rfm = model_results(forecasted_rfm, current_version)

        if RunType == 'Forward':
        
            try:
            # generating feature importance
                feat_imp_df_cbr = feat_imp(model=cbr, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=True, cat=cat)
                feat_imp_df_lgbr = feat_imp(model=lgbr, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=True, cat=cat)
                feat_imp_df_rfm = feat_imp(model=rfm, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=True, cat=cat)
                feat_imp_df_xgboostr = feat_imp(model=xgboostr, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=True, cat=cat)

                                                                                                            
                concat_grain = ['Version.[Version Name]','Time.[Week]','Item.[L6]','Sales Domain.[Sales Org]','[ML Driver].[ML Driver]']
                feat_imp_df_cbr = feat_imp_df_cbr.set_index(concat_grain)
                feat_imp_df_lgbr = feat_imp_df_lgbr.set_index(concat_grain)
                feat_imp_df_rfm = feat_imp_df_rfm.set_index(concat_grain)
                feat_imp_df_xgboostr = feat_imp_df_xgboostr.set_index(concat_grain)

                feat_imp_df = pd.concat([feat_imp_df_cbr,feat_imp_df_lgbr,feat_imp_df_rfm,feat_imp_df_xgboostr],axis=1).reset_index()

            except:
                feat_imp_df = pd.DataFrame()
                logger.debug('No feat_imp_df generated')                


    # MLL modeling
    if len(mll_df) != 0:
        logger.debug("MLL Model running")

        mll_df.fillna(0, inplace=True)
        mll_df = mll_df.sort_values(by=['PPG', 'Planning_Entity', 'SalesOrg', 'WEEK_NUM']).reset_index(drop=True)

        remove = ['Shipment_Volume', 'PPG', 'WEEK_NUM', 'Planning_Entity', 'SalesOrg',
                  'Distribution', 'DisplayOnlyPrice',
                  'FeatureOnlyPrice', 'FeatureandDisplayPrice',
                  'PriceReductionPrice', 'StandardPrice', 'DISPLAY_ONLY_PROMO_FLAG',
                  'FEATURE_DISPLAY_PROMO_FLAG', 'FEATURE_ONLY_PROMO_FLAG',
                  'PRICE_REDUCTIONS_ONLY_PROMO_FLAG', 'DISPLAY_ONLY_PROMO_FLAG_lead2',
                  'DISPLAY_ONLY_PROMO_FLAG_lead3', 'FEATURE_DISPLAY_PROMO_FLAG_lead2',
                  'FEATURE_DISPLAY_PROMO_FLAG_lead3', 'FEATURE_ONLY_PROMO_FLAG_lead2',
                  'FEATURE_ONLY_PROMO_FLAG_lead3',
                  'PRICE_REDUCTIONS_ONLY_PROMO_FLAG_lead2',
                  'PRICE_REDUCTIONS_ONLY_PROMO_FLAG_lead3', 'weekno', 'weights', 'Year']

        partial_remove = ['Shipment_Volume',
                  'Distribution', 'DisplayOnlyPrice',
                  'FeatureOnlyPrice', 'FeatureandDisplayPrice',
                  'PriceReductionPrice', 'StandardPrice', 'DISPLAY_ONLY_PROMO_FLAG',
                  'FEATURE_DISPLAY_PROMO_FLAG', 'FEATURE_ONLY_PROMO_FLAG',
                  'PRICE_REDUCTIONS_ONLY_PROMO_FLAG', 'DISPLAY_ONLY_PROMO_FLAG_lead2',
                  'DISPLAY_ONLY_PROMO_FLAG_lead3', 'FEATURE_DISPLAY_PROMO_FLAG_lead2',
                  'FEATURE_DISPLAY_PROMO_FLAG_lead3', 'FEATURE_ONLY_PROMO_FLAG_lead2',
                  'FEATURE_ONLY_PROMO_FLAG_lead3',
                  'PRICE_REDUCTIONS_ONLY_PROMO_FLAG_lead2',
                  'PRICE_REDUCTIONS_ONLY_PROMO_FLAG_lead3', 'weekno', 'weights', 'Year']

        if RunType == 'Forward':
            weeklist = sorted(i for i in mll_df['WEEK_NUM'].unique().tolist() if i > ship_max_week)[:-4]
        elif RunType == 'Backtest':
            weeklist = [sorted(i for i in mll_df['WEEK_NUM'].unique().tolist() if i > ship_max_week)[3]]
        logger.debug("Weeks to be forecasted for MLL")
        logger.debug(weeklist)

        forecasted_xgb = []
        forecasted_cat = []
        forecasted_lgb = []
        forecasted_rfm = []

        train = mll_df[(mll_df['WEEK_NUM'] >= starting_week) & (mll_df['WEEK_NUM'] <= ship_max_week)]
        test = mll_df[(mll_df['WEEK_NUM'] > ship_max_week)]

        train['IsTrain'] = 1
        test['IsTrain'] = 0
        MLL_ADS = pd.concat([train, test], axis=0, ignore_index=True)
        train = train.drop(['IsTrain'], axis=1)
        test = test.drop(['IsTrain'], axis=1)
        #######
        try:
            MLL_ADS = pd.concat([ML_ADS, MLL_ADS], axis=0, ignore_index=True)
        except:
            logger.debug('no drive data for ' + cat)

        #######
        for i in MLL_ADS.columns:
            MLL_ADS[i] = MLL_ADS[i].astype(str)

        logger.debug('Writing MLL+ML df')
        n = MLL_ADS.shape[0]
        logger.debug('Number of records to write:' + str(n))
        start = time.time()
        upload_ads_to_azure(MLL_ADS, connect_str,
                            tenant_name + '/' + Folder_name + Parquet_subfolder_prefix + str(ship_max_week) + '/ML_fwd_' + str(cat) + '_US',
                            destination_name)
        logger.debug('time taken to write in parquet format : ' + str(time.time() - start))

        ls1 = train['Planning_Entity'].unique().tolist()
        test = test[test['Planning_Entity'].isin(ls1)]

        ls2 = train['PPG'].unique().tolist()
        test = test[test['PPG'].isin(ls2)]

        dummy1 = pd.get_dummies(train['PPG'])
        dummy2 = pd.get_dummies(train['Planning_Entity'])

        # dropping zero columns
        try:
            dummy1.drop([0], axis=1, inplace=True)
            dummy2.drop([0], axis=1, inplace=True)
        except:
            logger.debug("No zero column in dummy variables")

        train = pd.concat([train, dummy1, dummy2], axis=1)

        train.replace(np.inf, np.nan, inplace=True)
        train.fillna(0, inplace=True)

        try:
            train.drop([0], axis=1, inplace=True)
        except:
            logger.debug("No zero column in train data")

        dummy1 = pd.get_dummies(test['PPG'])
        dummy2 = pd.get_dummies(test['Planning_Entity'])

        # dropping zero columns
        try:
            dummy1.drop([0], axis=1, inplace=True)
            dummy2.drop([0], axis=1, inplace=True)
        except:
            logger.debug("No zero column in dummy variables")

        test = pd.concat([test, dummy1, dummy2], axis=1)

        test.replace(np.inf, np.nan, inplace=True)
        test.fillna(0, inplace=True)

        try:
            test.drop([0], axis=1, inplace=True)
        except:
            logger.debug("No zero column in test data")

        cols = test.columns.to_list()
        train = train[cols]

        # creating recency based model weights on training set
        le = preprocessing.LabelEncoder()
        logger.debug('Encoder input:')
        logger.debug(train.WEEK_NUM.values)
        train['weekno'] = le.fit_transform(train.WEEK_NUM.values)
        logger.debug('Encoder output:')
        logger.debug(train['weekno'].head())
        train["weights"] = 30 + (train["weekno"] - train["weekno"].max()) * np.exp(-2)
        train["weights"] = train["weights"].astype(float)
        train['weights'] = train['weights'] / train['weights'].max()
        train.loc[(train.WEEK_NUM > 202005) & (train.WEEK_NUM < 202028), 'weights'] = 0.2
        train.loc[(train.WEEK_NUM > 202010) & (train.WEEK_NUM < 202020), 'weights'] = 0.05

        train_y1 = train['Shipment_Volume']
        train_x1 = train[train.columns[~train.columns.isin(remove)]]

        # 1 Xgboost Model
        xgboostr = XGBRegressor(random_state=2017)
        xgboostr.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        logger.debug('Xgboost Fitting completed successfully')

        # 2 Catboost Model
        cbr = CatBoostRegressor(random_state=2017)
        cbr.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        # 3 lgbm Model
        lgbr = LGBMRegressor(random_state=2017)
        lgbr.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        # 4 rfm Model
        rfm = RandomForestRegressor(random_state=2017)
        rfm.fit(train_x1, train_y1, sample_weight=train["weights"].to_list())

        models_MLL = {'xgboostr_driver_less': xgboostr, 'cbr_driver_less': cbr, 'lgbr_driver_less': lgbr,
                      'rfm_driver_less': rfm}
        if 'models' in locals():
            models_MLL = {**models,**models_MLL}

        if Pickle == 'Yes':
            start = time.time()
            upload_pickle_to_azure(models_MLL, connect_str, tenant_name+ '/' + Folder_name + Pickle_subfolder_prefix + str(
                ship_max_week) + '/models_' + cat + '_US'+'.pkl', destination_name)
            logger.debug('time taken to write in pickle file : ' + str(time.time() - start))

        logger.debug('Fitting completed successfully')

        try:

            explainer_cbr_driverless = shap.TreeExplainer(cbr)
            explainer_lgbr_driverless = shap.TreeExplainer(lgbr)
            explainer_rfm_driverless = shap.TreeExplainer(rfm)
            explainer_xgboostr_driverless = shap.TreeExplainer(xgboostr)

            explainers_driverless = {'xgboostr_driver_less': explainer_xgboostr_driverless, 'cbr_driver_less': explainer_cbr_driverless, 'lgbr_driver_less': explainer_lgbr_driverless,
                          'rfm_driver_less': explainer_rfm_driverless}

            start = time.time()
            upload_pickle_to_azure(explainers_driverless, connect_str, tenant_name+ '/' + Folder_name + Pickle_subfolder_prefix + str(
                ship_max_week) + '/explainers_driverless_' + cat + '_US'+'.pkl', destination_name)
            logger.debug('time taken to write in pickle file : ' + str(time.time() - start))

            test_x = test[test.columns[~test.columns.isin(partial_remove)]]

            start = time.time()
            upload_ads_to_azure(test_x, connect_str,
                                tenant_name + '/' + Folder_name + Parquet_subfolder_prefix + str(ship_max_week) + '/ML_fwd_' + str(cat) + '_US_test_MLL',
                                destination_name)
            logger.debug('time taken to write in parquet format : ' + str(time.time() - start))

        except:
            logger.debug('explainer_driverless failed to generate')

        predict_start_time = time.time()

        logger.debug(weeklist)
        for i in weeklist:
            logger.debug('i')
            logger.debug(i)

            test_1 = test[test['WEEK_NUM'] == i]

            test_y1 = test_1['Shipment_Volume']
            test_x1 = test_1[test_1.columns[~test_1.columns.isin(remove)]]

            train_x1 = train_x1.fillna(0)
            test_x1 = test_x1.fillna(0)

            cols = train_x1.columns.to_list()
            test_x1 = test_x1[cols]

            logger.debug("week: {}".format(i))
            # XGBoost prediction
            pred_xgb = xgboostr.predict(test_x1)
            pred_xgb[pred_xgb < 0] = 0

            testdata_xgb = test_1.copy()
            testdata_xgb['Predicted'] = pred_xgb
            testdata_xgb['Time.[Week]'] = i
            testdata_xgb = testdata_xgb[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_xgb.append(testdata_xgb)

            # Catboost prediction
            pred_cat = cbr.predict(test_x1)
            pred_cat[pred_cat < 0] = 0

            testdata_cat = test_1.copy()
            testdata_cat['Predicted'] = pred_cat
            testdata_cat['Time.[Week]'] = i
            testdata_cat = testdata_cat[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_cat.append(testdata_cat)

            # lgbm prediction
            pred_lgb = lgbr.predict(test_x1)
            pred_lgb[pred_lgb < 0] = 0

            testdata_lgb = test_1.copy()
            testdata_lgb['Predicted'] = pred_lgb
            testdata_lgb['Time.[Week]'] = i
            testdata_lgb = testdata_lgb[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_lgb.append(testdata_lgb)

            # Random forest prediction
            pred_rfm = rfm.predict(test_x1)
            pred_rfm[pred_rfm < 0] = 0

            testdata_rfm = test_1.copy()
            testdata_rfm['Predicted'] = pred_rfm
            testdata_rfm['Time.[Week]'] = i
            testdata_rfm = testdata_rfm[['WEEK_NUM', 'PPG', 'Planning_Entity', 'Time.[Week]', 'SalesOrg', 'Predicted']]

            forecasted_rfm.append(testdata_rfm)
            
        logger.debug('Predictions generated successfully')

        predict_end_time = time.time()
        predict_time = predict_end_time - predict_start_time
        logger.debug('predict_time (MLL): {}'.format(predict_time))

        # Generating Xgboost output #
        df_results_xgb1 = model_results(forecasted_xgb, current_version)
        # Generating Catboost output #
        df_results_cat1 = model_results(forecasted_cat, current_version)
        # Generating LGBM output #
        df_results_lgb1 = model_results(forecasted_lgb, current_version)
        # Generating RFM output #
        df_results_rfm1 = model_results(forecasted_rfm, current_version)
            
        if RunType == 'Forward':
        
            try:
            
                # Generating FI for MLL mode
                feat_imp_df1_cbr = feat_imp(model=cbr, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=False, cat=cat)
                feat_imp_df1_lgbr = feat_imp(model=lgbr, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=False, cat=cat)
                feat_imp_df1_rfm = feat_imp(model=rfm, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=False, cat=cat)
                feat_imp_df1_xgboostr = feat_imp(model=xgboostr, train_x1=train_x1, current_version=current_version,
                                       ship_max_week=ship_max_week, featflag=False, cat=cat)

                concat_grain = ['Version.[Version Name]','Time.[Week]','Item.[L6]','Sales Domain.[Sales Org]','[ML Driver].[ML Driver]']
                feat_imp_df1_cbr = feat_imp_df1_cbr.set_index(concat_grain)
                feat_imp_df1_lgbr = feat_imp_df1_lgbr.set_index(concat_grain)
                feat_imp_df1_rfm = feat_imp_df1_rfm.set_index(concat_grain)
                feat_imp_df1_xgboostr = feat_imp_df1_xgboostr.set_index(concat_grain)

                feat_imp_df1 = pd.concat([feat_imp_df1_cbr,feat_imp_df1_lgbr,feat_imp_df1_rfm,feat_imp_df1_xgboostr],axis=1).reset_index()

            except:
                feat_imp_df1 = pd.DataFrame()
                logger.debug('feat_imp_df1 not generated')

    # Final result for Catboost (Model 1 in tenant) #
    if len(ml_df) != 0:
        logger.debug("ML+MLL Result")
        try:
            actual_results_cat = pd.concat([df_results_cat, df_results_cat1], ignore_index=True)
        except:
            actual_results_cat = df_results_cat

        actual_results_cat = actual_results_cat.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_cat.drop(['SalesOrg'], axis=1, inplace=True)

        if RunType == 'Forward':
            actual_results_cat.rename(columns={'Predicted': 'ML Model 1 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_cat.rename(columns={'Predicted': 'ML Model 1 Fcst HL Lag2'}, inplace=True)
        actual_model1 = actual_results_cat.copy()

        # Final result for LGBM (Model 2 in tenant) #
        try:
            actual_results_lgb = pd.concat([df_results_lgb, df_results_lgb1], ignore_index=True)
        except:
            actual_results_lgb = df_results_lgb

        actual_results_lgb = actual_results_lgb.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_lgb.drop(['SalesOrg'], axis=1, inplace=True)

        if RunType == 'Forward':
            actual_results_lgb.rename(columns={'Predicted': 'ML Model 2 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_lgb.rename(columns={'Predicted': 'ML Model 2 Fcst HL Lag2'}, inplace=True)
        actual_model2 = actual_results_lgb.copy()

        # Final result for Random Forest Model (Model 3 in tenant) #
        try:
            actual_results_rfm = pd.concat([df_results_rfm, df_results_rfm1], ignore_index=True)
        except:
            actual_results_rfm = df_results_rfm

        actual_results_rfm = actual_results_rfm.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_rfm.drop(['SalesOrg'], axis=1, inplace=True)

        if RunType == 'Forward':
            actual_results_rfm.rename(columns={'Predicted': 'ML Model 3 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_rfm.rename(columns={'Predicted': 'ML Model 3 Fcst HL Lag2'}, inplace=True)
        actual_model3 = actual_results_rfm.copy()

        # Final result for Xgboost Model (Model 4 in tenant) #
        try:
            actual_results_xgb = pd.concat([df_results_xgb, df_results_xgb1], ignore_index=True)
        except:
            actual_results_xgb = df_results_xgb

        actual_results_xgb = actual_results_xgb.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_xgb.drop(['SalesOrg'], axis=1, inplace=True)

        if RunType == 'Forward':
            actual_results_xgb.rename(columns={'Predicted': 'ML Model 4 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_xgb.rename(columns={'Predicted': 'ML Model 4 Fcst HL Lag2'}, inplace=True)
        actual_model4 = actual_results_xgb.copy()
            
        if RunType == 'Forward':
            return actual_model1, actual_model2, actual_model3, actual_model4, feat_imp_df
        else:
            return actual_model1, actual_model2, actual_model3, actual_model4       

    else:
        logger.debug("FS Result")
        actual_results_cat = df_results_cat1

        actual_results_cat = actual_results_cat.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_cat.drop(['SalesOrg'], axis=1, inplace=True)
        if RunType == 'Forward':
            actual_results_cat.rename(columns={'Predicted': 'ML Model 1 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_cat.rename(columns={'Predicted': 'ML Model 1 Fcst HL Lag2'}, inplace=True)
        actual_model1 = actual_results_cat.copy()

        # Final result for LGBM (Model 2 in tenant) #
        actual_results_lgb = df_results_lgb1

        actual_results_lgb = actual_results_lgb.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_lgb.drop(['SalesOrg'], axis=1, inplace=True)
        if RunType == 'Forward':
            actual_results_lgb.rename(columns={'Predicted': 'ML Model 2 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_lgb.rename(columns={'Predicted': 'ML Model 2 Fcst HL Lag2'}, inplace=True)
        actual_model2 = actual_results_lgb.copy()

        # Final result for Random Forest Model (Model 3 in tenant) #
        actual_results_rfm = df_results_rfm1

        actual_results_rfm = actual_results_rfm.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_rfm.drop(['SalesOrg'], axis=1, inplace=True)
        if RunType == 'Forward':
            actual_results_rfm.rename(columns={'Predicted': 'ML Model 3 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_rfm.rename(columns={'Predicted': 'ML Model 3 Fcst HL Lag2'}, inplace=True)
        actual_model3 = actual_results_rfm.copy()

        # Final result for Xgboost Model (Model 4 in tenant) #
        actual_results_xgb = df_results_xgb1

        actual_results_xgb = actual_results_xgb.groupby(['Version.[Version Name]', 'Item.[L2]',
                                                         'Sales Domain.[Customer Group]', 'Time.[Week]', 'SalesOrg'])[
            'Predicted'].sum().reset_index().drop_duplicates()

        actual_results_xgb.drop(['SalesOrg'], axis=1, inplace=True)
        if RunType == 'Forward':
            actual_results_xgb.rename(columns={'Predicted': 'ML Model 4 Fcst HL'}, inplace=True)
        elif RunType == 'Backtest':
            actual_results_xgb.rename(columns={'Predicted': 'ML Model 4 Fcst HL Lag2'}, inplace=True)
        actual_model4 = actual_results_xgb.copy()
        
        if RunType == 'Forward':
            return actual_model1, actual_model2, actual_model3, actual_model4, feat_imp_df1
        else:
            return actual_model1, actual_model2, actual_model3, actual_model4 


ml_df, mll_df, starting_week, ship_max_week, ship_min_week = ads(shipment_df=shipment_df,
                                                                 imputed_price_promo_df=imputed_price_promo_df,
                                                                                                   
                                                                 year_week_df=time_df, calendar_df=calendar_df,
                                                                 uom_df=uom_df, compass_ppg=compass_ppg)

if RunType=='Forward':
    result1, result2, result3, result4, feat = model_run(ml_df=ml_df, mll_df=mll_df, starting_week=starting_week,
                                                              ship_max_week=ship_max_week, ship_min_week=ship_min_week,
                                                              current_version=current_version, cat=cat)
else:
    result1, result2, result3, result4 = model_run(ml_df=ml_df, mll_df=mll_df, starting_week=starting_week,
                                                              ship_max_week=ship_max_week, ship_min_week=ship_min_week,
                                                              current_version=current_version, cat=cat)

if RunType == 'Forward':
    logger.debug(result1.groupby('Time.[Week]')['ML Model 1 Fcst HL'].sum().astype(np.int))
    logger.debug(result2.groupby('Time.[Week]')['ML Model 2 Fcst HL'].sum().astype(np.int))
    logger.debug(result3.groupby('Time.[Week]')['ML Model 3 Fcst HL'].sum().astype(np.int))
    logger.debug(result4.groupby('Time.[Week]')['ML Model 4 Fcst HL'].sum().astype(np.int))
elif RunType == 'Backtest':
    logger.debug(result1.groupby('Time.[Week]')['ML Model 1 Fcst HL Lag2'].sum().astype(np.int))
    logger.debug(result2.groupby('Time.[Week]')['ML Model 2 Fcst HL Lag2'].sum().astype(np.int))
    logger.debug(result3.groupby('Time.[Week]')['ML Model 3 Fcst HL Lag2'].sum().astype(np.int))
    logger.debug(result4.groupby('Time.[Week]')['ML Model 4 Fcst HL Lag2'].sum().astype(np.int))

logger.debug("Shape of Model 1 result")
logger.debug(result1.shape)
logger.debug("Sum of Model 1 predicted result")
if RunType == 'Forward':
    logger.debug(result1['ML Model 1 Fcst HL'].sum())
elif RunType == 'Backtest':
    logger.debug(result1['ML Model 1 Fcst HL Lag2'].sum())
logger.debug("Shape of Model 2 result")
logger.debug(result2.shape)
logger.debug("Shape of Model 3 result")
logger.debug(result2.shape)
logger.debug("Shape of Model 4 result")
logger.debug(result4.shape)

Actual_model = reduce(lambda x, y: pd.merge(x, y,
                                            on=['Version.[Version Name]', 'Item.[L2]', 'Sales Domain.[Customer Group]',
                                                'Time.[Week]'], how='left'), [result1, result2, result3, result4])

Feat_model = pd.DataFrame()                                
if RunType == 'Forward':
    try:
        Feat_model = feat
        logger.debug(Feat_model['Driver Importance Model 1'].sum())
    except:
        logger.debug('No feat output generated')
    

logger.debug("Actual_model.Shape")
logger.debug(Actual_model.shape)
logger.debug("Actual_model.head()")
logger.debug(Actual_model.head())

###################################################
logger.debug("KHC ML Fwd-Backtest script completed...!")

ModuleNotFoundError: No module named 'xgboost'